In [30]:
import statsmodels.api as sm 
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.graphics import tsaplots
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.stattools import durbin_watson
from sklearn.metrics import mean_absolute_error

def excel_to_pandas(path):
    """This function converts a csv file into a pandas dataframe. 
    
    Parameters:
        Path: Where the csv file is located on the device
    
    Returns a pandas dataframe.
    """
    New = pd.read_excel(path)
    return New 

**1. Calling in the combined Yahoo Finance and Google Trends data**

In [20]:
Trends_and_Price = excel_to_pandas('/Users/jacobkuhlman/Desktop/Final Project DA /Google Trends and Price/Trends_and_Price_Year.xlsx')

**2. Extracting just the Monero data**

In [21]:
Monero_Trends = Trends_and_Price.loc[Trends_and_Price['Cryptocurrency_type'] == 'Monero']
Monero_Trends = Monero_Trends[['Date','Close', 'Search Volume', 'Volume']]
Monero_Trends = Monero_Trends.set_index('Date')
Monero_Trends = Monero_Trends.fillna(method = 'pad')

**3. Creating a training and testing set**

In [22]:
n_obs=15
#n_obs=50
X_train, X_test = Monero_Trends[0:-n_obs], Monero_Trends[-n_obs:]
print(X_train.shape, X_test.shape)

(1753, 3) (15, 3)


**4. Conducting the ADF test**

In [33]:
X_train_transformed = X_train.diff().dropna()
transform_data = X_train.diff().dropna()

def augmented_dickey_fuller_statistics(time_series):
    result = adfuller(time_series.values)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

        
#print('Augmented Dickey-Fuller Test: Close Price Time Series')
#augmented_dickey_fuller_statistics(X_train_transformed['Close'])

#print('Augmented Dickey-Fuller Test: Search Volume Time Series')
#augmented_dickey_fuller_statistics(X_train_transformed['Search Volume'])

Augmented Dickey-Fuller Test: Close Price Time Series
ADF Statistic: -8.571359
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568
Augmented Dickey-Fuller Test: Search Volume Time Series
ADF Statistic: -17.900557
p-value: 0.000000
Critical Values:
	1%: -3.434
	5%: -2.863
	10%: -2.568


**5. Conducting the Granger Causality test**

In [24]:
maxlag = 12
test = 'ssr-chi2test'
def grangers_causality_matrix(X_train, variables, test = 'ssr_chi2test', verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables) 
    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(X_train[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value
            #dataset.columns = [var + '_x' for var in variables]
            #dataset.index = [var + '_y' for var in variables]
    return dataset
grangers_causality_matrix(Monero_Trends, variables = Monero_Trends.columns)

,Close,Search Volume,Volume
Close,1.0000,0.1213,0.4566
Search Volume,0.0000,1.0000,0.0000
Volume,0.0111,0.0227,1.0000
